In [234]:
import pandas as pd
import numpy as np
from mystuff import bdp,bdh
import datetime
import funciones_optim
from importlib import reload
reload(funciones_optim)
from funciones_optim import backt

In [235]:
# carga precios de bloomberg 
p = backt.loadprices(20171231)

In [236]:
#calculo de retornos para cada asset class
r = p.apply(backt.rt,args=[p],axis=0).drop(['USCASH'],axis=1)
r = r.loc[:,['SPX','EEM','XAU','USTTEN','COCASH','USDCOP']]

In [237]:
#rango de fechas donde se va a hacer el backtesting
sdate = datetime.datetime(2019,1,31)
fdate = datetime.datetime(2023,12,31)

#Ejemplo de un portafolio equally weighted que rebalancea mensualmente y vuelve siempre al vector inicial
w = np.ones(6)*0.20
rdates =pd.date_range(sdate,fdate)
rbw = pd.DataFrame(np.tile(w,(len(rdates),1)),index=rdates).resample('M').last()
wp,rp = backt.portr(r,sdate,fdate,w,rebal=False)

In [238]:
# ejemplo de 5 portafolios con rebalanceos mensuales a recomendaciones ( en este caso los w son generados aleateoriamente como proxy de un modelo)
# wi es el vector inicial del modelo
# rbw contiene los vectores recomendacion mensuales
ports=[]
for p in range(5):
    wx = pd.DataFrame(np.random.rand(6,len(rbw)))
    wx = wx/wx.iloc[:-1,:].sum()
    wx = wx.transpose()
    wx.index = rbw.index
    wi = np.random.rand(6)
    wi = wi/sum(wi[:-1])
    w_,r_ = backt.portr(r,sdate,fdate,wi,rebal=True,rbw=wx)
    ports.append(r_)
ports = pd.concat(ports,axis=1)
ports.columns = [f'port {c}' for c in ports.columns]


In [239]:
# devuelve un dataframe como metricas de desempeño y riesgo de los portafolios.
df = backt.pmetrics(ports,r.COCASH,bench=0)

In [240]:
#un ejemplo de dataframe comparativo en metricas de portafolio. 
df.round(3)

,port 0,port 1,port 2,port 3,port 4
ret,30.040,35.097,40.909,24.390,51.623
retm,0.464,0.535,0.602,0.391,0.730
volm,2.176,2.758,2.466,2.336,2.365
sharpe,-0.021,0.009,0.037,-0.051,0.093
mdd,-16.963,-17.153,-15.062,-14.068,-16.102
etg/etl,1.253,1.105,1.472,1.502,1.527
downdev,2.032,2.557,2.032,1.942,2.346
sortino,0.228,0.209,0.296,0.202,0.311
